In [1]:
%tensorflow_version 1.x
import tensorflow as tf
tf.__version__


TensorFlow 1.x selected.


'1.15.2'

In [2]:
!pip install laspy

     |████████████████████████████████| 491kB 2.5MB/s 


In [3]:
# load data
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.chdir("/content/gdrive/My Drive/ColabNotebooks/IMF")

In [0]:

import sys
import tensorflow as tf
import numpy as np
import math
import timeit
import scipy

#import matplotlib.pyplot as plt

import provider
import TR
#%matplotlib inline

In [6]:
USE_GPU = True

if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

# Constant to control how often we print when training models
print_every = 100

print('Using device: ', device)

Using device:  /device:GPU:0


In [0]:
from mobilenet_v3_block import BottleNeck, h_swish

class CentralNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2,channel_3,channel_4, NUM_CLASSES,num_points):
        super().__init__()
        initializer = tf.variance_scaling_initializer(scale=2.0)

        # mobilenet
        self.conv1 = tf.keras.layers.Conv2D(filters=16,kernel_size=(3, 3),strides=2,padding="same")
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.bneck1 = BottleNeck(in_size=16, exp_size=16, out_size=16, s=2, is_se_existing=True, NL="RE", k=3)
        self.bneck2 = BottleNeck(in_size=16, exp_size=72, out_size=24, s=2, is_se_existing=False, NL="RE", k=3)
        self.bneck3 = BottleNeck(in_size=24, exp_size=88, out_size=24, s=1, is_se_existing=False, NL="RE", k=3)
        self.bneck4 = BottleNeck(in_size=24, exp_size=96, out_size=40, s=2, is_se_existing=True, NL="HS", k=5)
        self.bneck5 = BottleNeck(in_size=40, exp_size=240, out_size=40, s=1, is_se_existing=True, NL="HS", k=5)
        self.bneck6 = BottleNeck(in_size=40, exp_size=240, out_size=40, s=1, is_se_existing=True, NL="HS", k=5)
        self.bneck7 = BottleNeck(in_size=40, exp_size=120, out_size=48, s=1, is_se_existing=True, NL="HS", k=5)
        self.bneck8 = BottleNeck(in_size=48, exp_size=144, out_size=48, s=1, is_se_existing=True, NL="HS", k=5)
        self.bneck9 = BottleNeck(in_size=48, exp_size=288, out_size=96, s=2, is_se_existing=True, NL="HS", k=5)
        self.bneck10 = BottleNeck(in_size=96, exp_size=576, out_size=96, s=1, is_se_existing=True, NL="HS", k=5)
        self.bneck11 = BottleNeck(in_size=96, exp_size=576, out_size=96, s=1, is_se_existing=True, NL="HS", k=5)

        self.conv2 = tf.keras.layers.Conv2D(filters=576,kernel_size=(1, 1),strides=1,padding="same")
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.avgpool = tf.keras.layers.AveragePooling2D(pool_size=(1, 1),strides=1)
        self.conv3 = tf.keras.layers.Conv2D(filters=1280,
                                            kernel_size=(1, 1),
                                            strides=1,
                                            padding="same")
        self.conv4 = tf.keras.layers.Conv2D(filters=NUM_CLASSES,
                                            kernel_size=(1, 1),
                                            strides=1,
                                            padding="same",
                                            activation=tf.keras.activations.softmax)
        ###pointnet
        self.convpn1 = tf.layers.Conv2D(64, kernel_size=(1,3),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-512
        self.convpn2 = tf.layers.Conv2D(64, kernel_size=(1,1),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-512
        self.maxpoolpn1 = tf.layers.MaxPooling2D(pool_size = (num_points,1),strides=(16,1), padding='SAME')   

        self.convpn3 = tf.layers.Conv2D(64, kernel_size=(1,1),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-512
          
        self.convpn4 = tf.layers.Conv2D(128, kernel_size=(1,1),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-512
        self.convpn5 = tf.layers.Conv2D(256, kernel_size=(1,1),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-512
        self.maxpoolpn = tf.layers.MaxPooling2D(pool_size = (num_points,1),strides=(num_points,1), padding='SAME')
        # fusion
                                     
        self.fc4c1 = tf.layers.Dense(channel_1, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        
        self.c14c2 = tf.layers.Dense(channel_2, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.fc4c2 = tf.layers.Dense(channel_2, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.c24c3 = tf.layers.Dense(channel_3, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.fc4c3 = tf.layers.Dense(channel_3, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.c34c4 = tf.layers.Dense(channel_4, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.fc4c4 = tf.layers.Dense(channel_4, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.c44c5 = tf.layers.Dense(1280, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        
        self.fc4p1 = tf.layers.Dense(channel_1, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.fc4p2 = tf.layers.Dense(channel_2, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.fc4p3 = tf.layers.Dense(channel_3, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.fc4p4 = tf.layers.Dense(channel_4, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        
        self.flatten = tf.keras.layers.Flatten()
        
        self.fc1=tf.layers.Dense(1280, activation = tf.nn.relu, 
                        kernel_initializer=initializer)
        self.fp1=tf.layers.Dense(1280, activation = tf.nn.relu, 
                        kernel_initializer=initializer)
        self.fc2=tf.layers.Dense(1280, activation = tf.nn.relu, 
                        kernel_initializer=initializer)
        self.fc3=tf.layers.Dense(1000, activation = tf.nn.relu, 
                        kernel_initializer=initializer)
        self.fc4=tf.layers.Dense(num_classes, activation = None, 
                        kernel_initializer=initializer)
     
    def call(self, x1, point_cloud, training=None):
        scores = None
        batch_size = point_cloud.get_shape()[0].value
        num_point = point_cloud.get_shape()[1].value
        x2 = tf.expand_dims(point_cloud, -1)
        ###x1

        x1 = self.conv1(x1)
        x1 = self.bn1(x1, training=training)
        x1 = h_swish(x1)

        x1 = self.bneck1(x1, training=training)
        x1 = self.bneck2(x1, training=training)
        x1 = self.bneck3(x1, training=training)
        h1_x1=self.flatten(x1)
        x1 = self.bneck4(x1, training=training)
        x1 = self.bneck5(x1, training=training)
        x1 = self.bneck6(x1, training=training)
        h2_x1=self.flatten(x1)
        x1 = self.bneck7(x1, training=training)
        x1 = self.bneck8(x1, training=training)
        h3_x1=self.flatten(x1)
        x1 = self.bneck9(x1, training=training)
        x1 = self.bneck10(x1, training=training)
        x1 = self.bneck11(x1, training=training)
        h4_x1=self.flatten(x1)
        x1 = self.conv2(x1)
        x1 = self.bn2(x1, training=training)
        x1 = h_swish(x1)
        x1 = self.avgpool(x1)
        x1 = self.conv3(x1)
        x1 = h_swish(x1)
        x1 = self.conv4(x1)
        h5_x1=self.flatten(x1)
        ###x2

        x2 = self.convpn1(x2)
        h1_x2=self.flatten(x2)

        x2 = self.convpn2(x2)
        h2_x2=self.flatten(tf.layers.max_pooling2d(x2, pool_size = (num_points,1),strides=(16,1), padding='SAME'))
        x2 = self.convpn3(x2)
        h3_x2=self.flatten(tf.layers.max_pooling2d(x2,pool_size = (num_points,1),strides=(32,1), padding='SAME')) #8,10000x3x64
        x2 = self.convpn4(x2)
        h4_x2=self.flatten(tf.layers.max_pooling2d(x2,pool_size = (num_points,1),strides=(32,1), padding='SAME')) #8,10000x3x128
        x2 = self.convpn5(x2) #64x10000x3x256
        x2= self.maxpoolpn(x2) #64x1x3x256
        h5_x2=self.flatten(x2)#8,1x3x256

        hc1=self.fc4c1(h1_x1)+self.fc4p1(h1_x2)
        hc2=self.fc4c2(h2_x1)+self.fc4p2(h2_x2)+self.c14c2(hc1)
        hc3=self.fc4c3(h3_x1)+self.fc4p3(h3_x2)+self.c24c3(hc2)
        hc4=self.fc4c4(h4_x1)+self.fc4p4(h4_x2)+self.c34c4(hc3)
        hc5=self.fc1(h5_x1)+self.fp1(h5_x2)+self.c44c5(hc4)
        dense2=self.fc2(hc5) #1280
        dense3=self.fc3(dense2)#1000
        scores= self.fc4(dense3)#num_classes
        return scores


def placeholder_inputs_pc(batch_size, img_rows=66, img_cols=200, points=16384, separately=False):
    imgs_pl = tf.placeholder(tf.float32, shape=(batch_size, img_rows, img_cols, 3))
    pts_pl = tf.placeholder(tf.float32, shape=(batch_size, points, 3))
    if separately:
        speeds_pl = tf.placeholder(tf.float32, shape=(batch_size))
        angles_pl = tf.placeholder(tf.float32, shape=(batch_size))
        labels_pl = [speeds_pl, angles_pl]
    labels_pl = tf.placeholder(tf.float32, shape=(batch_size, 2))
    return imgs_pl, pts_pl, labels_pl


def get_loss(pred, label, l2_weight=0.0001):
    diff = tf.square(tf.subtract(pred, label))
    train_vars = tf.trainable_variables()
    l2_loss = tf.add_n([tf.nn.l2_loss(v) for v in train_vars[1:]]) * l2_weight
    loss = tf.reduce_mean(diff + l2_loss)

    return loss



In [0]:

tf.compat.v1.reset_default_graph()

channel_1, channel_2,channel_3,channel_4, num_classes,num_points=64,128,256,512,10,16384
model = CentralNet(channel_1, channel_2,channel_3,channel_4, num_classes,num_points)
with tf.device(device):
    x1 = tf.zeros((8, 66, 200,3))
    x2 = tf.zeros((8, num_points,3))
    scores = model(x1,x2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    scores_np = sess.run(scores)
    print(scores_np.shape)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Please use `layer.__call__` method instead.
(8, 10)


In [0]:
def train_part34(model_init_fn, optimizer_init_fn, MAX_EPOCH, device, BATCH_SIZE):

    tf.reset_default_graph()    
    with tf.device(device):

        data_input = provider.DVR_Points_Provider()

        imgs_pl, pts_pl, labels_pl = placeholder_inputs_pc(BATCH_SIZE)
        x1=imgs_pl
        x2=pts_pl
        y=labels_pl
        
        is_training = tf.placeholder(tf.bool, name='is_training')
        
        # Use the model function to build the forward pass.
        scores = model_init_fn(x1,x2, is_training)

        # Compute the loss like we did in Part II
        loss = get_loss(scores,y)
        
        optimizer = optimizer_init_fn()
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            train_op = optimizer.minimize(loss)
        
        # Add ops to save and restore all the variables
        saver = tf.train.Saver(tf.global_variables())
            

    # Now we can run the computational graph many times to train the model.
    # When we call sess.run we ask it to evaluate train_op, which causes the
    # model to update.
    # Create a session
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.allow_soft_placement = True
    config.log_device_placement = False
    sess = tf.Session(config=config)

    sess.run(tf.global_variables_initializer())
    
    ops = {'x1': x1,
            'x2': x2,
            'y': y,
            'is_training': is_training,
            'train_op': train_op,
            'scores': scores,
            'loss': loss}

    eval_acc_max = 0
    for epoch in range(MAX_EPOCH):
        print('Starting epoch %d' % epoch)
        #for x_np, y_np in train_dset:
        #    feed_dict = {x1: x_np,x2:x_np, y: y_np, is_training:1}
        #    loss_np, _ = sess.run([loss, train_op], feed_dict=feed_dict)
        #    if t % print_every == 0:
        #        print('Iteration %d, loss = %.4f' % (t, loss_np))
        #        check_accuracy(sess, val_dset, x1,x2, scores, is_training=is_training)
        #        print()
        #    t += 1
        train_one_epoch(sess, ops, data_input, BATCH_SIZE)
        eval_acc=eval_one_epoch(sess, ops, data_input, BATCH_SIZE)
        if eval_acc > eval_acc_max:
            eval_acc_max = eval_acc
            save_path = saver.save(sess, "log/model_best.ckpt")
            print("Model saved in file: %s" % save_path)

        # Save the variables to disk.
        if epoch % 10 == 0:
            save_path = saver.save(sess, 'log/model.ckpt')
            print("Model saved in file: %s" % save_path)

def train_one_epoch(sess, ops, data_input, BATCH_SIZE):
    """ ops: dict mapping from string to tf ops """
    is_training = True
    num_batches = data_input.num_train // BATCH_SIZE
    loss_sum = 0
    acc_a_sum = 0
    acc_s_sum = 0

    for batch_idx in range(num_batches):

        imgs, pts,labels = data_input.load_one_batch(BATCH_SIZE, "train")
        feed_dict = {ops['x1']: imgs,
                     ops['x2']: pts,
                     ops['y']: labels,
                     ops['is_training']: is_training}

        _, loss_val, pred_val = sess.run([ops['train_op'],
                                         ops['loss'],
                                         ops['scores']],
                                        feed_dict=feed_dict)

        loss_sum, acc_a_sum, acc_s_sum=check_accuracy(loss_sum, acc_a_sum, acc_s_sum,pred_val,labels)

    print('mean loss: %f' % (loss_sum / float(num_batches)))
    print('accuracy (angle): %f' % (acc_a_sum / float(num_batches)))
    print('accuracy (speed): %f' % (acc_s_sum / float(num_batches)))


def eval_one_epoch(sess, ops, data_input, BATCH_SIZE):
    """ ops: dict mapping from string to tf ops """
    is_training = False
    loss_sum = 0

    num_batches = data_input.num_val // BATCH_SIZE
    loss_sum = 0
    acc_a_sum = 0
    acc_s_sum = 0

    for batch_idx in range(num_batches):
        imgs, pts,labels = data_input.load_one_batch(BATCH_SIZE, "val")
        feed_dict = {ops['x1']: imgs,
                     ops['x2']: pts,
                     ops['y']: labels,
                     ops['is_training']: is_training}

        _, loss_val, pred_val = sess.run([ops['train_op'],
                                         ops['loss'],
                                         ops['scores']],
                                        feed_dict=feed_dict)
        print(pred_val[10:20,1],labels[10:20,1])
        print(pred_val[10:20,0],labels[10:20,0])

        loss_sum, acc_a_sum, acc_s_sum=check_accuracy(loss_sum, acc_a_sum, acc_s_sum,pred_val,labels)

    print('eval mean loss: %f' % (loss_sum / float(num_batches)))
    print('eval accuracy (angle): %f' % (acc_a_sum / float(num_batches)))
    print('eval accuracy (speed): %f' % (acc_s_sum / float(num_batches)))
    return acc_a_sum / float(num_batches)

def  check_accuracy(loss_sum, acc_a_sum, acc_s_sum,pred_val,labels):
    loss_sum += np.mean(np.square(np.subtract(pred_val, labels)))
    
    acc_a = np.abs(np.subtract(pred_val[:, 1], labels[:, 1])) < (5.0 / 180 * scipy.pi)

    acc_a = np.mean(acc_a)
    acc_a_sum += acc_a
    acc_s = np.abs(np.subtract(pred_val[:, 0], labels[:, 0])) < (5.0 / 20)

    acc_s = np.mean(acc_s)
    acc_s_sum += acc_s
    return loss_sum, acc_a_sum, acc_s_sum

In [9]:
BATCH_SIZE=4
MAX_EPOCH=1
learning_rate = 3e-6
channel_1, channel_2,channel_3,channel_4, num_classes,num_points=64,128,256,512,2,16384

def model_init_fn(inputs1,inputs2, is_training):
    model = None

    model = CentralNet(channel_1, channel_2,channel_3,channel_4, num_classes,num_points)

    return model(inputs1,inputs2)

def optimizer_init_fn():
    optimizer = None

    optimizer = tf.train.AdamOptimizer()

    return optimizer

train_part34(model_init_fn, optimizer_init_fn,1,device,BATCH_SIZE)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Starting epoch 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:135: RuntimeWarning: invalid value encountered in less
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:139: RuntimeWarning: invalid value encountered in less


mean loss: nan
accuracy (angle): 0.000000
accuracy (speed): 0.000000
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[